
### What's different here:
* Data frames are made differently here (e.g. average rating no longer takes zero-ratings into account)
* __averageRatingdf__ is now called __books_rated__
* __ratings__ is now called __readers_15plus__
* __books_ratings__ has all the ratings aggregated in the new columns, so no user_id, and is saved to books_rated.csv
* __books_users_ratings__ has all the aggregated ratings columns as well as user_id and individual ratings, saved to books_users_ratings.csv
* __books_rated__ is then joined again with __ratings__ (right join) to have all the info needed for collaborative filtering

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import pickle
# import sklearn
# import pickleshare as ps

In [3]:
%run import_data

Continuing with existing version of data folder
Goodreads dataset loaded successfully as books_goodreads
Pandas dataframes (books_goodreads, books_big, book, users, ratings) loaded successfully
Columns in DataFrames 'users' and 'ratings' renamed
You can use the DataFrames 'books' or 'books_big' - they are exactly the same (big) dataset
loading books_ratings and books_users_ratings
Ready to go!


## I. Creating A Search Engine

#### Reducing shape of the dataframe ####

In [4]:
raters_15plus = books_ratings[books_ratings['nr_ratings'] >= 15]    #filtering out books with less than 15 readers
raters_15plus.shape

(3094, 14)

In [5]:
# raters_15plus.head()

,isbn,book_title,book_author,year_of_publication,publisher,genre,avg_rating,nr_ratings,nr_readers,annotations,mod_title,image_url_s,image_url_m,image_url_l
0,971880107,Wild Animus,Rich Shapero,2004,Too Far,"['Fiction', 'Other']",4.390706,581,2504,NaN,wild animus,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...
1,316666343,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown","['Fiction', 'Other']",8.185290,707,1295,New York Times bestseller.Familles de victimes...,the lovely bones a novel,http://images.amazon.com/images/P/0316666343.0...,http://images.amazon.com/images/P/0316666343.0...,http://images.amazon.com/images/P/0316666343.0...
2,385504209,The Da Vinci Code,Dan Brown,2003,Doubleday,"['Fiction', 'Historical Fiction', 'Other', 'Th...",8.437628,489,885,New York Times reviewed.The Priory of Sion —a ...,the da vinci code,http://images.amazon.com/images/P/0385504209.0...,http://images.amazon.com/images/P/0385504209.0...,http://images.amazon.com/images/P/0385504209.0...
3,60928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,Perennial,"['Fiction', 'Other']",7.887500,320,732,NaN,divine secrets of the ya ya sisterhood a novel,http://images.amazon.com/images/P/0060928336.0...,http://images.amazon.com/images/P/0060928336.0...,http://images.amazon.com/images/P/0060928336.0...
4,312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA,"['Fiction', 'Other']",8.182292,384,724,NaN,the red tent bestselling backlist,http://images.amazon.com/images/P/0312195516.0...,http://images.amazon.com/images/P/0312195516.0...,http://images.amazon.com/images/P/0312195516.0...


In [6]:
raters_15plus.to_csv("data/raters_15plus.csv", sep=";", encoding="utf-8-sig", index=False)

#### Building a Term Frequency and then a Inverse Document Frequency matrix ####


In [222]:
# turning titles into TD-IDF matrix => Term Frequency-Inverse Document Frequency
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer takes a list of strings as input and turns it into a fd-idf matrix
vectorizer = TfidfVectorizer()

tdidf = vectorizer.fit_transform(readers_15plus['mod_title'])

In [223]:
# to do comparison between books, we need to calculate the cosine similarity between the books

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re


# query function:

def search(query, vectorizer):
# setting up a search query

    #query = 'The Hobbit'
# preparing the string in the same way as the mod titles above
    processed = re.sub('[^a-zA-Z0-9]', ' ', query.lower())

# we need to turn the query into a vector using the vectorizer

    query_vector = vectorizer.transform([processed])

# to find the similarities we calculate the cosine similarity between the query vector and the tdidf matrix
    similarity = cosine_similarity(query_vector, tdidf).flatten() # flatten is used to turn the matrix into a 1D array

# to find the indices of the 10 largest similarities

    indices = np.argpartition(similarity, -10)[-10:]

# use indices to index the titles

    results = readers_15plus.iloc[indices]   

# as there are many books with the title "The Hobbit", we only want those with the highest number of ratings

    results = results.sort_values(by='nr_ratings', ascending=False)

    return results.head(10)


In [224]:
search('the hitchhikers guide to the galaxy', vectorizer)

,isbn,book_title,book_author,year_of_publication,publisher,genre,avg_rating,nr_ratings,nr_readers,annotations,mod_title,image_url_s,image_url_m,image_url_l
50,140293248,The Girls' Guide to Hunting and Fishing,Melissa Bank,2000,Penguin Books,"['Fiction', 'Other', 'Romance']",6.909091,165,367,NaN,the girls guide to hunting and fishing,http://images.amazon.com/images/P/0140293248.0...,http://images.amazon.com/images/P/0140293248.0...,http://images.amazon.com/images/P/0140293248.0...
326,67088300X,The Girls' Guide to Hunting and Fishing,Melissa Bank,1999,Viking Books,"['Fiction', 'Other', 'Romance']",6.959459,74,157,NaN,the girls guide to hunting and fishing,http://images.amazon.com/images/P/067088300X.0...,http://images.amazon.com/images/P/067088300X.0...,http://images.amazon.com/images/P/067088300X.0...
445,1878424319,The Four Agreements: A Practical Guide to Pers...,Don Miguel Ruiz,1997,Amber-Allen Publishing,"['Self-Help', 'Other']",7.500000,62,133,New York Times bestseller,the four agreements a practical guide to perso...,http://images.amazon.com/images/P/1878424319.0...,http://images.amazon.com/images/P/1878424319.0...,http://images.amazon.com/images/P/1878424319.0...
1059,375504613,A Short Guide to a Happy Life,ANNA QUINDLEN,2000,Random House,"['Fiction', 'Other']",7.883721,43,78,NaN,a short guide to a happy life,http://images.amazon.com/images/P/0375504613.0...,http://images.amazon.com/images/P/0375504613.0...,http://images.amazon.com/images/P/0375504613.0...
1284,671527215,Hitchhikers's Guide to the Galaxy,Douglas Adams,1984,Pocket,"['Fiction', 'Science', 'Other']",8.576923,26,69,Far out in the uncharted backwaters of the unf...,hitchhikers s guide to the galaxy,http://images.amazon.com/images/P/0671527215.0...,http://images.amazon.com/images/P/0671527215.0...,http://images.amazon.com/images/P/0671527215.0...
3187,517149257,The Ultimate Hitchhiker's Guide,Douglas Adams,1996,Random House Value Publishing,"['Fiction', 'Science', 'Other']",9.045455,22,36,NaN,the ultimate hitchhiker s guide,http://images.amazon.com/images/P/0517149257.0...,http://images.amazon.com/images/P/0517149257.0...,http://images.amazon.com/images/P/0517149257.0...
3309,671524313,The Girlfriends' Guide to Pregnancy,Vicki Iovine,1995,Pocket,['Other'],7.650000,20,35,NaN,the girlfriends guide to pregnancy,http://images.amazon.com/images/P/0671524313.0...,http://images.amazon.com/images/P/0671524313.0...,http://images.amazon.com/images/P/0671524313.0...
2728,671461494,The Hitchhiker's Guide to the Galaxy,Douglas Adams,1982,Pocket,"['Fiction', 'Science', 'Other']",8.277778,18,41,Far out in the uncharted backwaters of the unf...,the hitchhiker s guide to the galaxy,http://images.amazon.com/images/P/0671461494.0...,http://images.amazon.com/images/P/0671461494.0...,http://images.amazon.com/images/P/0671461494.0...
4834,811828964,The Bad Girl's Guide to Getting What You Want,Cameron Tuttle,2000,Chronicle Books,"['Psychology', 'Other']",6.812500,16,26,NaN,the bad girl s guide to getting what you want,http://images.amazon.com/images/P/0811828964.0...,http://images.amazon.com/images/P/0811828964.0...,http://images.amazon.com/images/P/0811828964.0...
5359,345453743,The Ultimate Hitchhiker's Guide to the Galaxy,DOUGLAS ADAMS,2002,Del Rey,"['Fiction', 'Science', 'Other']",8.875000,16,24,NaN,the ultimate hitchhiker s guide to the galaxy,http://images.amazon.com/images/P/0345453743.0...,http://images.amazon.com/images/P/0345453743.0...,http://images.amazon.com/images/P/0345453743.0...


## II. Making Recommendations Based On A User's Liked Books

In [225]:
# creating my list of liked books
liked_books = ['006440546X', '1929462379', '1853263451', '1891575139', '9505811322', '1561840718']
liked_books

['006440546X',
 '1929462379',
 '1853263451',
 '1891575139',
 '9505811322',
 '1561840718']

#### step 1: find all the users that liked the same books as us ####

In [226]:
raters_15plus.head()

,isbn,book_title,book_author,year_of_publication,publisher,genre,avg_rating,nr_ratings,nr_readers,annotations,mod_title,image_url_s,image_url_m,image_url_l
0,971880107,Wild Animus,Rich Shapero,2004,Too Far,"['Fiction', 'Other']",4.390706,581,2504,NaN,wild animus,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...
1,316666343,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown","['Fiction', 'Other']",8.185290,707,1295,New York Times bestseller.Familles de victimes...,the lovely bones a novel,http://images.amazon.com/images/P/0316666343.0...,http://images.amazon.com/images/P/0316666343.0...,http://images.amazon.com/images/P/0316666343.0...
2,385504209,The Da Vinci Code,Dan Brown,2003,Doubleday,"['Fiction', 'Historical Fiction', 'Other', 'Th...",8.437628,489,885,New York Times reviewed.The Priory of Sion —a ...,the da vinci code,http://images.amazon.com/images/P/0385504209.0...,http://images.amazon.com/images/P/0385504209.0...,http://images.amazon.com/images/P/0385504209.0...
3,60928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,Perennial,"['Fiction', 'Other']",7.887500,320,732,NaN,divine secrets of the ya ya sisterhood a novel,http://images.amazon.com/images/P/0060928336.0...,http://images.amazon.com/images/P/0060928336.0...,http://images.amazon.com/images/P/0060928336.0...
4,312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA,"['Fiction', 'Other']",8.182292,384,724,NaN,the red tent bestselling backlist,http://images.amazon.com/images/P/0312195516.0...,http://images.amazon.com/images/P/0312195516.0...,http://images.amazon.com/images/P/0312195516.0...


In [227]:
filtered_ratings = books_users_ratings[books_users_ratings['isbn'].isin(liked_books) & (books_users_ratings['individual_rating'] > 8)]  #filtering out books with ratings higher or equal  8   

print(filtered_ratings.shape)

# create a set with users who liked the same books as me

overlap_users = set()

# # Create tuples (user_id, isbn, book_rating) for the filtered rows
overlap_users = set(filtered_ratings.apply(lambda row: (row['user_id'], row['isbn'],row['book_title'], row['individual_rating']), axis=1))

# # Display the overlap_users set
overlap_users

(9, 16)


{(56399.0,
  '1853263451',
  "Culpeper's Complete Herbal: A Book of Natural Remedies of Ancient Ills (The Wordsworth Collection Reference Library)",
  10.0),
 (76626.0,
  '1853263451',
  "Culpeper's Complete Herbal: A Book of Natural Remedies of Ancient Ills (The Wordsworth Collection Reference Library)",
  10.0),
 (97324.0,
  '9505811322',
  'LA Metamorfosis (Colecci?on Literaria Lyc (Leer y Crear))',
  10.0),
 (141310.0,
  '1561840718',
  'Quantum Psychology: How Brain Software Programs You and Your World',
  10.0),
 (141322.0,
  '1561840718',
  'Quantum Psychology: How Brain Software Programs You and Your World',
  10.0),
 (173660.0,
  '1561840718',
  'Quantum Psychology: How Brain Software Programs You and Your World',
  10.0),
 (187810.0,
  '9505811322',
  'LA Metamorfosis (Colecci?on Literaria Lyc (Leer y Crear))',
  10.0),
 (265591.0,
  '9505811322',
  'LA Metamorfosis (Colecci?on Literaria Lyc (Leer y Crear))',
  10.0),
 (273718.0,
  '1853263451',
  "Culpeper's Complete Herbal:

In [228]:
# making set a dataframe containing any user who read the same book as us and rated the book higher than 8
overlap_users_df = pd.DataFrame(list(overlap_users), columns=['user_id', 'isbn', 'book_title', 'individual_rating'])
overlap_users_df

,user_id,isbn,book_title,individual_rating
0,76626.0,1853263451,Culpeper's Complete Herbal: A Book of Natural ...,10.0
1,141322.0,1561840718,Quantum Psychology: How Brain Software Program...,10.0
2,265591.0,9505811322,LA Metamorfosis (Colecci?on Literaria Lyc (Lee...,10.0
3,187810.0,9505811322,LA Metamorfosis (Colecci?on Literaria Lyc (Lee...,10.0
4,173660.0,1561840718,Quantum Psychology: How Brain Software Program...,10.0
5,56399.0,1853263451,Culpeper's Complete Herbal: A Book of Natural ...,10.0
6,273718.0,1853263451,Culpeper's Complete Herbal: A Book of Natural ...,10.0
7,97324.0,9505811322,LA Metamorfosis (Colecci?on Literaria Lyc (Lee...,10.0
8,141310.0,1561840718,Quantum Psychology: How Brain Software Program...,10.0


In [229]:
len(overlap_users_df)

9

In [230]:
overlap_users_df['isbn'] = overlap_users_df['isbn'].astype(str)   #converting isbn to string

#### step 2: finding what those users liked ####

In [231]:
# finding books that the users in the overlap_users_df have read and rated highly

#filtering out books read by users in the overlap_users_df
total_books = books_users_ratings[books_users_ratings['user_id'].isin(overlap_users_df['user_id'])]

#filtering out books with ratings higher than 8
total_books = total_books[total_books['individual_rating'] > 8]

#filtering out books that I have already liked
recommended = total_books[~total_books['isbn'].isin(liked_books)]

#finding the top 10 books that the users in the overlap_users_df have read and rated highly
recommended_2 = recommended['isbn'].value_counts().head(10)

In [232]:
len(recommended_2)

10

In [233]:
total_books.head()

,isbn,book_title,book_author,user_id,individual_rating,avg_rating,nr_ratings,nr_readers,genre,year_of_publication,publisher,annotations,mod_title,image_url_s,image_url_m,image_url_l
15,1903436346,The Winter's Tale - Arden Shakespeare : Third ...,John Pitcher,76626.0,9.0,9.000000,1,1,"['Fiction', 'Historical Fiction', 'Other']",2006,Arden,British and irish drama (dramatic works by one...,the winter s tale arden shakespeare third seri...,http://images.amazon.com/images/P/1903436346.0...,http://images.amazon.com/images/P/1903436346.0...,http://images.amazon.com/images/P/1903436346.0...
129,140144463,The Cornish Trilogy,Robertson Davies,76626.0,10.0,9.000000,2,2,"['Fiction', 'Historical Fiction', 'Other']",2005,Penguin Books Ltd,Canadian fiction (fictional works by one author),the cornish trilogy,http://images.amazon.com/images/P/0140144463.0...,http://images.amazon.com/images/P/0140144463.0...,http://images.amazon.com/images/P/0140144463.0...
3980,670032891,The Well of Lost Plots: A Thursday Next Novel,Jasper Fforde,76626.0,10.0,8.538462,13,38,"['Fiction', 'Other']",2004,Viking Books,Thursday Next (Fictitious character),the well of lost plots a thursday next novel,http://images.amazon.com/images/P/0670032891.0...,http://images.amazon.com/images/P/0670032891.0...,http://images.amazon.com/images/P/0670032891.0...
5921,859914976,The Green Man,Kathleen Basford,56399.0,10.0,10.000000,1,1,['Other'],2004,D.S.Brewer,NaN,the green man,http://images.amazon.com/images/P/0859914976.0...,http://images.amazon.com/images/P/0859914976.0...,http://images.amazon.com/images/P/0859914976.0...
8128,1561841625,The Earth Will Shake (Historical Illuminatus C...,Robert Anton Wilson,56399.0,10.0,10.000000,1,1,"['Science', 'Fiction', 'Historical Fiction', '...",2004,New Falcon Publications,NaN,the earth will shake historical illuminatus ch...,http://images.amazon.com/images/P/1561841625.0...,http://images.amazon.com/images/P/1561841625.0...,http://images.amazon.com/images/P/1561841625.0...


In [234]:
total_books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 975 entries, 15 to 917367
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   isbn                 975 non-null    object 
 1   book_title           975 non-null    object 
 2   book_author          975 non-null    object 
 3   user_id              975 non-null    float64
 4   individual_rating    975 non-null    float64
 5   avg_rating           975 non-null    float64
 6   nr_ratings           975 non-null    int64  
 7   nr_readers           975 non-null    int64  
 8   genre                975 non-null    object 
 9   year_of_publication  975 non-null    int64  
 10  publisher            975 non-null    object 
 11  annotations          376 non-null    object 
 12  mod_title            975 non-null    object 
 13  image_url_s          975 non-null    object 
 14  image_url_m          975 non-null    object 
 15  image_url_l          975 non-null    obje

In [235]:
recommended_2.head()

isbn
1561840564    3
1561840033    3
553572857     2
553212621     2
55357289X     2
Name: count, dtype: int64

In [236]:
# Convert recommended_2 Series to DataFrame
recommended_2_df = recommended_2.reset_index()
recommended_2_df.columns = ['isbn', 'count']

In [237]:
recommended_2_df

,isbn,count
0,1561840564,3
1,1561840033,3
2,553572857,2
3,553212621,2
4,55357289X,2
5,875428681,2
6,140143505,2
7,156001314,2
8,553572881,2
9,64471098,2


In [238]:
# Merge with books DataFrame to get book titles
recommended_with_titles = pd.merge(recommended_2_df, total_books[['isbn', 'book_title']], on='isbn', how='left')

# Display the result
recommended_with_titles[['isbn', 'book_title']].value_counts().reset_index(name='count')

,isbn,book_title,count
0,1561840033,Cosmic Trigger I : Final Secret of the Illuminati,3
1,1561840564,Prometheus Rising,3
2,140143505,84 Charing Cross Road,2
3,156001314,The Name of the Rose: including Postscript to ...,2
4,553212621,Lady Chatterley's Lover: Complete and Unexpurg...,2
5,553572857,"Shadow Moon (Chronicles of the Shadow War, Boo...",2
6,553572881,"Shadow Star (Chronicles of the Shadow War, Boo...",2
7,55357289X,"Shadow Dawn (Chronicles of the Shadow War, Boo...",2
8,64471098,The Silver Chair,2
9,875428681,The Magician's Companion: A Practical and Ency...,2


#### step 3: fine tuning the recommendations ####

In [239]:
total_books.head()

,isbn,book_title,book_author,user_id,individual_rating,avg_rating,nr_ratings,nr_readers,genre,year_of_publication,publisher,annotations,mod_title,image_url_s,image_url_m,image_url_l
15,1903436346,The Winter's Tale - Arden Shakespeare : Third ...,John Pitcher,76626.0,9.0,9.000000,1,1,"['Fiction', 'Historical Fiction', 'Other']",2006,Arden,British and irish drama (dramatic works by one...,the winter s tale arden shakespeare third seri...,http://images.amazon.com/images/P/1903436346.0...,http://images.amazon.com/images/P/1903436346.0...,http://images.amazon.com/images/P/1903436346.0...
129,140144463,The Cornish Trilogy,Robertson Davies,76626.0,10.0,9.000000,2,2,"['Fiction', 'Historical Fiction', 'Other']",2005,Penguin Books Ltd,Canadian fiction (fictional works by one author),the cornish trilogy,http://images.amazon.com/images/P/0140144463.0...,http://images.amazon.com/images/P/0140144463.0...,http://images.amazon.com/images/P/0140144463.0...
3980,670032891,The Well of Lost Plots: A Thursday Next Novel,Jasper Fforde,76626.0,10.0,8.538462,13,38,"['Fiction', 'Other']",2004,Viking Books,Thursday Next (Fictitious character),the well of lost plots a thursday next novel,http://images.amazon.com/images/P/0670032891.0...,http://images.amazon.com/images/P/0670032891.0...,http://images.amazon.com/images/P/0670032891.0...
5921,859914976,The Green Man,Kathleen Basford,56399.0,10.0,10.000000,1,1,['Other'],2004,D.S.Brewer,NaN,the green man,http://images.amazon.com/images/P/0859914976.0...,http://images.amazon.com/images/P/0859914976.0...,http://images.amazon.com/images/P/0859914976.0...
8128,1561841625,The Earth Will Shake (Historical Illuminatus C...,Robert Anton Wilson,56399.0,10.0,10.000000,1,1,"['Science', 'Fiction', 'Historical Fiction', '...",2004,New Falcon Publications,NaN,the earth will shake historical illuminatus ch...,http://images.amazon.com/images/P/1561841625.0...,http://images.amazon.com/images/P/1561841625.0...,http://images.amazon.com/images/P/1561841625.0...


In [240]:
#creating a column book_count  
recommended = recommended.copy() # this to handle the warning from pandas
recommended.loc[:, 'book_count'] = recommended.groupby('isbn')['isbn'].transform('count')
recommended.head()

,isbn,book_title,book_author,user_id,individual_rating,avg_rating,nr_ratings,nr_readers,genre,year_of_publication,publisher,annotations,mod_title,image_url_s,image_url_m,image_url_l,book_count
15,1903436346,The Winter's Tale - Arden Shakespeare : Third ...,John Pitcher,76626.0,9.0,9.000000,1,1,"['Fiction', 'Historical Fiction', 'Other']",2006,Arden,British and irish drama (dramatic works by one...,the winter s tale arden shakespeare third seri...,http://images.amazon.com/images/P/1903436346.0...,http://images.amazon.com/images/P/1903436346.0...,http://images.amazon.com/images/P/1903436346.0...,1
129,140144463,The Cornish Trilogy,Robertson Davies,76626.0,10.0,9.000000,2,2,"['Fiction', 'Historical Fiction', 'Other']",2005,Penguin Books Ltd,Canadian fiction (fictional works by one author),the cornish trilogy,http://images.amazon.com/images/P/0140144463.0...,http://images.amazon.com/images/P/0140144463.0...,http://images.amazon.com/images/P/0140144463.0...,1
3980,670032891,The Well of Lost Plots: A Thursday Next Novel,Jasper Fforde,76626.0,10.0,8.538462,13,38,"['Fiction', 'Other']",2004,Viking Books,Thursday Next (Fictitious character),the well of lost plots a thursday next novel,http://images.amazon.com/images/P/0670032891.0...,http://images.amazon.com/images/P/0670032891.0...,http://images.amazon.com/images/P/0670032891.0...,1
5921,859914976,The Green Man,Kathleen Basford,56399.0,10.0,10.000000,1,1,['Other'],2004,D.S.Brewer,NaN,the green man,http://images.amazon.com/images/P/0859914976.0...,http://images.amazon.com/images/P/0859914976.0...,http://images.amazon.com/images/P/0859914976.0...,1
8128,1561841625,The Earth Will Shake (Historical Illuminatus C...,Robert Anton Wilson,56399.0,10.0,10.000000,1,1,"['Science', 'Fiction', 'Historical Fiction', '...",2004,New Falcon Publications,NaN,the earth will shake historical illuminatus ch...,http://images.amazon.com/images/P/1561841625.0...,http://images.amazon.com/images/P/1561841625.0...,http://images.amazon.com/images/P/1561841625.0...,1


In [241]:
recommended.book_count.unique()

array([1, 2, 3])

In [242]:
recommended.head()

,isbn,book_title,book_author,user_id,individual_rating,avg_rating,nr_ratings,nr_readers,genre,year_of_publication,publisher,annotations,mod_title,image_url_s,image_url_m,image_url_l,book_count
15,1903436346,The Winter's Tale - Arden Shakespeare : Third ...,John Pitcher,76626.0,9.0,9.000000,1,1,"['Fiction', 'Historical Fiction', 'Other']",2006,Arden,British and irish drama (dramatic works by one...,the winter s tale arden shakespeare third seri...,http://images.amazon.com/images/P/1903436346.0...,http://images.amazon.com/images/P/1903436346.0...,http://images.amazon.com/images/P/1903436346.0...,1
129,140144463,The Cornish Trilogy,Robertson Davies,76626.0,10.0,9.000000,2,2,"['Fiction', 'Historical Fiction', 'Other']",2005,Penguin Books Ltd,Canadian fiction (fictional works by one author),the cornish trilogy,http://images.amazon.com/images/P/0140144463.0...,http://images.amazon.com/images/P/0140144463.0...,http://images.amazon.com/images/P/0140144463.0...,1
3980,670032891,The Well of Lost Plots: A Thursday Next Novel,Jasper Fforde,76626.0,10.0,8.538462,13,38,"['Fiction', 'Other']",2004,Viking Books,Thursday Next (Fictitious character),the well of lost plots a thursday next novel,http://images.amazon.com/images/P/0670032891.0...,http://images.amazon.com/images/P/0670032891.0...,http://images.amazon.com/images/P/0670032891.0...,1
5921,859914976,The Green Man,Kathleen Basford,56399.0,10.0,10.000000,1,1,['Other'],2004,D.S.Brewer,NaN,the green man,http://images.amazon.com/images/P/0859914976.0...,http://images.amazon.com/images/P/0859914976.0...,http://images.amazon.com/images/P/0859914976.0...,1
8128,1561841625,The Earth Will Shake (Historical Illuminatus C...,Robert Anton Wilson,56399.0,10.0,10.000000,1,1,"['Science', 'Fiction', 'Historical Fiction', '...",2004,New Falcon Publications,NaN,the earth will shake historical illuminatus ch...,http://images.amazon.com/images/P/1561841625.0...,http://images.amazon.com/images/P/1561841625.0...,http://images.amazon.com/images/P/1561841625.0...,1


In [243]:
# creatig a popularity score for each book / we want to look for books that are popular among users like us
recommended = recommended.copy() # this to handle the pandas warning
recommended.loc[:, 'popularity'] = recommended['book_count'] * recommended['book_count'] / recommended['individual_rating']
recommended.head()

,isbn,book_title,book_author,user_id,individual_rating,avg_rating,nr_ratings,nr_readers,genre,year_of_publication,publisher,annotations,mod_title,image_url_s,image_url_m,image_url_l,book_count,popularity
15,1903436346,The Winter's Tale - Arden Shakespeare : Third ...,John Pitcher,76626.0,9.0,9.000000,1,1,"['Fiction', 'Historical Fiction', 'Other']",2006,Arden,British and irish drama (dramatic works by one...,the winter s tale arden shakespeare third seri...,http://images.amazon.com/images/P/1903436346.0...,http://images.amazon.com/images/P/1903436346.0...,http://images.amazon.com/images/P/1903436346.0...,1,0.111111
129,140144463,The Cornish Trilogy,Robertson Davies,76626.0,10.0,9.000000,2,2,"['Fiction', 'Historical Fiction', 'Other']",2005,Penguin Books Ltd,Canadian fiction (fictional works by one author),the cornish trilogy,http://images.amazon.com/images/P/0140144463.0...,http://images.amazon.com/images/P/0140144463.0...,http://images.amazon.com/images/P/0140144463.0...,1,0.100000
3980,670032891,The Well of Lost Plots: A Thursday Next Novel,Jasper Fforde,76626.0,10.0,8.538462,13,38,"['Fiction', 'Other']",2004,Viking Books,Thursday Next (Fictitious character),the well of lost plots a thursday next novel,http://images.amazon.com/images/P/0670032891.0...,http://images.amazon.com/images/P/0670032891.0...,http://images.amazon.com/images/P/0670032891.0...,1,0.100000
5921,859914976,The Green Man,Kathleen Basford,56399.0,10.0,10.000000,1,1,['Other'],2004,D.S.Brewer,NaN,the green man,http://images.amazon.com/images/P/0859914976.0...,http://images.amazon.com/images/P/0859914976.0...,http://images.amazon.com/images/P/0859914976.0...,1,0.100000
8128,1561841625,The Earth Will Shake (Historical Illuminatus C...,Robert Anton Wilson,56399.0,10.0,10.000000,1,1,"['Science', 'Fiction', 'Historical Fiction', '...",2004,New Falcon Publications,NaN,the earth will shake historical illuminatus ch...,http://images.amazon.com/images/P/1561841625.0...,http://images.amazon.com/images/P/1561841625.0...,http://images.amazon.com/images/P/1561841625.0...,1,0.100000


In [244]:
tailored_recs = recommended.sort_values('popularity', ascending=False)
tailored_recs.head()

,isbn,book_title,book_author,user_id,individual_rating,avg_rating,nr_ratings,nr_readers,genre,year_of_publication,publisher,annotations,mod_title,image_url_s,image_url_m,image_url_l,book_count,popularity
703504,1561840033,Cosmic Trigger I : Final Secret of the Illuminati,Robert A. Wilson,56399.0,10.0,9.500000,4,6,"['Fiction', 'Other']",1991,New Falcon Publications,NaN,cosmic trigger i final secret of the illuminati,http://images.amazon.com/images/P/1561840033.0...,http://images.amazon.com/images/P/1561840033.0...,http://images.amazon.com/images/P/1561840033.0...,3,0.9
671393,1561840564,Prometheus Rising,Robert Anton Wilson,141310.0,10.0,9.666667,6,6,['Other'],1992,New Falcon Publications,NaN,prometheus rising,http://images.amazon.com/images/P/1561840564.0...,http://images.amazon.com/images/P/1561840564.0...,http://images.amazon.com/images/P/1561840564.0...,3,0.9
703506,1561840033,Cosmic Trigger I : Final Secret of the Illuminati,Robert A. Wilson,141310.0,10.0,9.500000,4,6,"['Fiction', 'Other']",1991,New Falcon Publications,NaN,cosmic trigger i final secret of the illuminati,http://images.amazon.com/images/P/1561840033.0...,http://images.amazon.com/images/P/1561840033.0...,http://images.amazon.com/images/P/1561840033.0...,3,0.9
671395,1561840564,Prometheus Rising,Robert Anton Wilson,173660.0,10.0,9.666667,6,6,['Other'],1992,New Falcon Publications,NaN,prometheus rising,http://images.amazon.com/images/P/1561840564.0...,http://images.amazon.com/images/P/1561840564.0...,http://images.amazon.com/images/P/1561840564.0...,3,0.9
703507,1561840033,Cosmic Trigger I : Final Secret of the Illuminati,Robert A. Wilson,141322.0,10.0,9.500000,4,6,"['Fiction', 'Other']",1991,New Falcon Publications,NaN,cosmic trigger i final secret of the illuminati,http://images.amazon.com/images/P/1561840033.0...,http://images.amazon.com/images/P/1561840033.0...,http://images.amazon.com/images/P/1561840033.0...,3,0.9


In [245]:
tailored_recs[['isbn', 'book_title', 'popularity']].value_counts().reset_index(name='count').head(10)

,isbn,book_title,popularity,count
0,1561840033,Cosmic Trigger I : Final Secret of the Illuminati,0.900000,3
1,1561840564,Prometheus Rising,0.900000,3
2,517124858,The Ultimate Hitchhiker's Guide: Complete and ...,0.400000,2
3,156001314,The Name of the Rose: including Postscript to ...,0.444444,2
4,875428681,The Magician's Companion: A Practical and Ency...,0.400000,2
5,877285160,Book of Lies,0.400000,2
6,6440188X,The Secret Garden,0.100000,1
7,64404455,Tom's Midnight Garden,0.100000,1
8,64405028,The Voyage of the 'Dawn Treader' (The Chronicl...,0.100000,1
9,64407675,The Reptile Room (A Series of Unfortunate Even...,0.111111,1
